In [132]:
!pip install pandas
!pip install matplotlib
!pip install keras
!pip install wandb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [6]:
df = pd.read_pickle('dataset1.pkl')
df

,InitialCondition,Trajectory,Classification
0,"(-2.0, -1.4471544715447155, -1.099526161982576...","([-2.010936428329985, -2.0217547169615546, -2....",Bounded orbit
1,"(-2.0, -1.5447154471544715, -1.189881698919723...","([-2.0118383750574815, -2.023555345946939, -2....",Bounded orbit
2,"(-2.0, -0.21138211382113803, -0.13653797119674...","([-2.001322141896985, -2.00255775461015, -2.00...",Bounded orbit
3,"(-2.0, 0.5040650406504068, 0.3337940367874922,...","([-1.996616905942439, -1.993143643565462, -1.9...",Bounded orbit
4,"(-2.0, -0.9268292682926829, -0.650604284584492...","([-2.0064555976142824, -2.012810042355882, -2....",Bounded orbit
...,...,...,...
1995,"(-1.5121951219512195, 0.016260162601626327, 0....","([-1.5121251443240515, -1.512037597344692, -1....",Crash into primary 1
1996,"(-1.5121951219512195, -1.8048780487804879, -1....","([-1.5254269519443, -1.5385757534377793, -1.55...",Crash into primary 1
1997,"(-1.5121951219512195, 0.7317073170731709, 0.35...","([-1.508605601099163, -1.5049843456167733, -1....",Crash into primary 1
1998,"(-1.4796747967479673, -1.8373983739837398, -1....","([-1.4931627647983017, -1.506569316351737, -1....",Crash into primary 1


In [21]:
#check on the shape of the dataset, to find out along which dimension I should pad
trajectory_shapes = []

# Iterate over each trajectory and get its shape
for index, row in df.iterrows():
    trajectory = np.array(row['Trajectory'])
    trajectory_shapes.append(trajectory.shape)  # Append the shape to the list

# Now you have a list of the shapes of each trajectory
# You can convert this list to a DataFrame for easy inspection
shapes_df = pd.DataFrame(trajectory_shapes, columns=['Coordinates', 'Timesteps'])
filtered_df = shapes_df[shapes_df['Coordinates'] != 4]
filtered_df

,Coordinates,Timesteps


In [22]:
trajectory_tensors = [torch.tensor(t, dtype=torch.float32) for t in trajectories]

# Find the maximum number of features across all timesteps in all trajectories
max_timesteps = max(t.shape[1] for t in trajectory_tensors)

# Pad the trajectories along the features dimension and create masks
padded_trajectories = []
masks = []

for t in trajectory_tensors:
    # Calculate the padding needed for this trajectory
    padding_needed = max_timesteps - t.shape[1]
    
    # Pad the trajectory and store it
    padded = torch.nn.functional.pad(t, (0, padding_needed))
    padded_trajectories.append(padded)
    
    # Create the mask for this trajectory: True for actual data, False for padding
    mask = torch.ones_like(t, dtype=torch.bool)
    mask = torch.nn.functional.pad(mask, (0, padding_needed), value=False)
    masks.append(mask)

# Stack the padded trajectories and masks into tensors
padded_trajectories_tensor = torch.stack(padded_trajectories)
masks_tensor = torch.stack(masks)

In [27]:
padded_trajectories_tensor.shape

torch.Size([2000, 4, 9999])

In [28]:
#Model architecture
from torch import nn
import torch

def gaussian_init_(n_units, std=1):
    sampler = torch.distributions.Normal(torch.Tensor([0]), torch.Tensor([std/n_units]))
    Omega = sampler.sample((n_units, n_units))[..., 0]
    return Omega


class encoderNet(nn.Module):
    def __init__(self, m, n, b, ALPHA = 1):
        super(encoderNet, self).__init__()
        self.N = m * n
        self.tanh = nn.Tanh()

        self.fc1 = nn.Linear(self.N, 16*ALPHA)
        self.fc2 = nn.Linear(16*ALPHA, 16*ALPHA)
        self.fc3 = nn.Linear(16*ALPHA, b)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        x = x.view(-1, 1, self.N)
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.fc3(x)

        return x


class decoderNet(nn.Module):
    def __init__(self, m, n, b, ALPHA = 1):
        super(decoderNet, self).__init__()

        self.m = m
        self.n = n
        self.b = b

        self.tanh = nn.Tanh()

        self.fc1 = nn.Linear(b, 16*ALPHA)
        self.fc2 = nn.Linear(16*ALPHA, 16*ALPHA)
        self.fc3 = nn.Linear(16*ALPHA, m*n)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0.0)

    def forward(self, x):
        x = x.view(-1, 1, self.b)
        x = self.tanh(self.fc1(x))
        x = self.tanh(self.fc2(x))
        x = self.tanh(self.fc3(x))
        x = x.view(-1, 1, self.m, self.n)
        return x



class dynamics(nn.Module):
    def __init__(self, b, init_scale):
        super(dynamics, self).__init__()
        self.dynamics = nn.Linear(b, b, bias=False)
        self.dynamics.weight.data = gaussian_init_(b, std=1)
        U, _, V = torch.svd(self.dynamics.weight.data)
        self.dynamics.weight.data = torch.mm(U, V.t()) * init_scale


    def forward(self, x):
        x = self.dynamics(x)
        return x


class dynamics_back(nn.Module):
    def __init__(self, b, omega):
        super(dynamics_back, self).__init__()
        self.dynamics = nn.Linear(b, b, bias=False)
        self.dynamics.weight.data = torch.pinverse(omega.dynamics.weight.data.t())

    def forward(self, x):
        x = self.dynamics(x)
        return x


class koopmanAE(nn.Module):
    def __init__(self, m, n, b, steps, steps_back, alpha = 1, init_scale=1):
        super(koopmanAE, self).__init__()
        self.steps = steps
        self.steps_back = steps_back

        self.encoder = encoderNet(m, n, b, ALPHA = alpha)
        self.dynamics = dynamics(b, init_scale)
        self.backdynamics = dynamics_back(b, self.dynamics)
        self.decoder = decoderNet(m, n, b, ALPHA = alpha)


    def forward(self, x, mode='forward'):
        out = []
        out_back = []
        z = self.encoder(x.contiguous())
        q = z.contiguous()


        if mode == 'forward':
            for _ in range(self.steps):
                q = self.dynamics(q)
                out.append(self.decoder(q))

            out.append(self.decoder(z.contiguous()))
            return out, out_back

        if mode == 'backward':
            for _ in range(self.steps_back):
                q = self.backdynamics(q)
                out_back.append(self.decoder(q))

            out_back.append(self.decoder(z.contiguous()))
            return out, out_back

In [83]:
print(padded_trajectories_tensor.shape[0])

2000


In [87]:
import numpy as np

# Add noise to the dataset
padded_trajectories_processed = padded_trajectories_tensor + np.random.standard_normal(padded_trajectories_tensor.shape)

# Rotate to high-dimensional space for each trajectory separately
num_trajectories = padded_trajectories_tensor.shape[0]

# Rotate to high-dimensional space
Q = np.random.standard_normal((64,4))
Q,_ = np.linalg.qr(Q)

processed_X = []    #processed X is a list of trajectories
for i in range(num_trajectories):   
    result = torch.matmul(padded_trajectories_processed[i].T, torch.tensor(Q.T))
    min_val = torch.min(result)
    max_val = torch.max(result)
    result = 2 * (result - min_val) / (max_val - min_val) - 1  
    processed_X.append(result)



In [88]:
print(processed_X[0].shape, len(processed_X))

torch.Size([9999, 64]) 2000


In [137]:
import torch
import torchvision
from torch import nn
from torch import autograd
from torch import optim
from torchvision import transforms, datasets
from torch.autograd import grad
from timeit import default_timer as timer

import matplotlib.pyplot as plt

import numpy as np


import torch.nn.init as init

def set_seed(seed=0):
    """Set one seed for reproducibility."""
    np.random.seed(seed)
    torch.manual_seed(seed)

def get_device():
    """Get a gpu if available."""
    if torch.cuda.device_count()>0:
        device = torch.device('cuda')
        print("Connected to a GPU")
    else:
        print("Using the CPU")
        device = torch.device('cpu')
    return device


def add_channels(X):
    if len(X.shape) == 2:
        return X.reshape(X.shape[0], 1, X.shape[1],1)

    elif len(X.shape) == 3:
        return X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])

    else:
        return "dimenional error"


def weights_init(m):
    if isinstance(m, nn.Linear):
        if m.weight is not None:
            init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0.0)

In [138]:
#the train method

import torch
from torch import nn
import numpy as np
import wandb



def train(model, train_loader, lr, weight_decay,
          lamb, num_epochs, learning_rate_change, epoch_update,
          nu=0.0, eta=0.0, backward=0, steps=1, steps_back=1, gradclip=1):
        # Initialize wandb project
    wandb.init(project="Koopman Autoencoder", name="training-run-name")
    
    # Log hyperparameters
    wandb.config.lr = lr
    wandb.config.weight_decay = weight_decay
    wandb.config.lamb = lamb
    wandb.config.num_epochs = num_epochs
    wandb.config.learning_rate_change = learning_rate_change
    wandb.config.epoch_update = epoch_update
    wandb.config.nu = nu
    wandb.config.eta = eta
    wandb.config.backward = backward
    wandb.config.steps = steps
    wandb.config.steps_back = steps_back
    wandb.config.gradclip = gradclip

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    device = get_device()


    def lr_scheduler(optimizer, epoch, lr_decay_rate=0.8, decayEpoch=[]):
                    """Decay learning rate by a factor of lr_decay_rate every lr_decay_epoch epochs"""
                    if epoch in decayEpoch:
                        for param_group in optimizer.param_groups:
                            param_group['lr'] *= lr_decay_rate
                        return optimizer
                    else:
                        return optimizer




    criterion = nn.MSELoss().to(device)


    epoch_hist = []
    loss_hist = []
    epoch_loss = []


    for epoch in range(num_epochs):
        #print(epoch)
        for batch_idx, data_list in enumerate(train_loader):
            #print("batch index", batch_idx)
            #print("data list total length" , len(data_list))
            #print("first item in data list length", data_list[0].shape)
            model.train()
            out, out_back = model(data_list[0].to(device), mode='forward')



            for k in range(steps):
                if k == 0:
                    loss_fwd = criterion(out[k], data_list[k+1].to(device))
                    #print("loss between", out[k].shape, data_list[k+1].shape)
                else:
                    loss_fwd += criterion(out[k], data_list[k+1].to(device))


            loss_identity = criterion(out[-1], data_list[0].to(device)) * steps


            loss_bwd = 0.0
            loss_consist = 0.0

            loss_bwd = 0.0
            loss_consist = 0.0

            if backward == 1:
                out, out_back = model(data_list[-1].to(device), mode='backward')


                for k in range(steps_back):

                    if k == 0:
                        loss_bwd = criterion(out_back[k], data_list[::-1][k+1].to(device))
                    else:
                        loss_bwd += criterion(out_back[k], data_list[::-1][k+1].to(device))


                A = model.dynamics.dynamics.weight
                B = model.backdynamics.dynamics.weight

                K = A.shape[-1]

                for k in range(1,K+1):
                    As1 = A[:,:k]
                    Bs1 = B[:k,:]
                    As2 = A[:k,:]
                    Bs2 = B[:,:k]

                    Ik = torch.eye(k).float().to(device)

                    if k == 1:
                        loss_consist = (torch.sum((torch.mm(Bs1, As1) - Ik)**2) + \
                                         torch.sum((torch.mm(As2, Bs2) - Ik)**2) ) / (2.0*k)
                    else:
                        loss_consist += (torch.sum((torch.mm(Bs1, As1) - Ik)**2) + \
                                         torch.sum((torch.mm(As2, Bs2)-  Ik)**2) ) / (2.0*k)





#                Ik = torch.eye(K).float().to(device)
#                loss_consist = (torch.sum( (torch.mm(A, B)-Ik )**2)**1 + \
#                                         torch.sum( (torch.mm(B, A)-Ik)**2)**1 )
#



            loss = loss_fwd + lamb * loss_identity +  nu * loss_bwd + eta * loss_consist

            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradclip) # gradient clip
            optimizer.step()

        # schedule learning rate decay
        # Log metrics
        wandb.log({"epoch": epoch, "loss": loss.item(), "loss_identity": loss_identity.item(), "loss_forward": loss_fwd.item()})
        lr_scheduler(optimizer, epoch, lr_decay_rate=learning_rate_change, decayEpoch=epoch_update)
        loss_hist.append(loss)
        epoch_loss.append(epoch)


        if (epoch) % 20 == 0:
                print('********** Epoche %s **********' %(epoch+1))

                print("loss identity: ", loss_identity.item())
                if backward == 1:
                    print("loss backward: ", loss_bwd.item())
                    print("loss consistent: ", loss_consist.item())
                print("loss forward: ", loss_fwd.item())
                print("loss sum: ", loss.item())


                epoch_hist.append(epoch+1)

                if hasattr(model.dynamics, 'dynamics'):
                    w, _ = np.linalg.eig(model.dynamics.dynamics.weight.data.cpu().numpy())
                    print(np.abs(w))


    if backward == 1:
        loss_consist = loss_consist.item()


    return model, optimizer, [epoch_hist, loss_fwd.item(), loss_consist]



In [ ]:
from torch.utils.data import DataLoader, Dataset

import torch.nn.init as init

########################################### Model Hyperparameters ##############################################

#Declaring some hyperparameters - default values
lamb = 1
steps = 100
lr =1e-2
bottleneck= 6
lr_update = [30, 200, 400, 500],
epochs = 600
batch = 64
steps_back = 8
wd = 0
lr_decay = 0.2
nu =1e-1
eta = 1e-2
backward = 0
gradclip = 0.05
pred_steps = 1000
m = 64
n= 1




##############################   TRAIN LOADERS ###########################################################
train_loaders = []
test_dataset_trajectories = []
#should be len(processed_X)
for i in range(100):
    Xtrain = processed_X[i][0:9999]
    Xtest = processed_X[i][6000:10000]
    print("X train length", Xtrain.shape)
    print("X test length", Xtest.shape)
    #test_dataset_trajectories.append(Xtest)
    Xtrain = add_channels(Xtrain)
    Xtest = add_channels(Xtest)
    Xtrain = Xtrain.float().contiguous()
    Xtest = Xtest.float().contiguous()
    trainDat = []
    start = 0
    for i in np.arange(steps,-1, -1):
        if i == 0:
            trainDat.append(Xtrain[start:].float())
        else:
            trainDat.append(Xtrain[start:-i].float())
        start += 1

    train_data = torch.utils.data.TensorDataset(*trainDat)
    del(trainDat)



    train_loader = DataLoader(dataset = train_data,
                              batch_size = batch,
                              shuffle = True)

    train_loaders.append(train_loader)



#==============================================================================
# Model
#==============================================================================

model = koopmanAE(m, n, bottleneck, steps, steps_back)


############################ Printing the Model Architecture  #############################################
#print('koopmanAE')




#model = torch.nn.DataParallel(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)


#==============================================================================
# Model summary
#==============================================================================
'''
print('**** Setup ****')
print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))
print('Total params: %.2fk' % (sum(p.numel() for p in model.parameters())/1000.0))
print('************')
print(model)
'''

#==============================================================================
# Start training - training separately on each trajectory
#==============================================================================

# Start training on each trajectory separately
for idx, train_loader in enumerate(train_loaders):
    print(f'Training on trajectory {idx + 1}')
    model, optimizer, epoch_hist = train(model, train_loader, lr, wd, lamb, epochs,
                                         lr_decay, lr_update, nu, eta, backward,
                                         steps, steps_back, gradclip)
    # Save the trained model after training on each trajectory
    torch.save(model.state_dict(), f'model_trained_on_trajectory_{idx + 1}.pkl')



X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size([9999, 64])
X test length torch.Size([3999, 64])
X train length torch.Size

Connected to a GPU
********** Epoche 1 **********
loss identity:  0.03639815375208855
loss forward:  1.624560832977295
loss sum:  1.660959005355835
[0.83404857 1.0106785  0.71270967 0.71270967 0.6719378  0.6719378 ]
********** Epoche 21 **********
loss identity:  0.021318979561328888
loss forward:  1.330122947692871
loss sum:  1.3514419794082642
[0.6784041  0.43134025 0.43134025 0.9992048  0.99262154 0.01942833]
********** Epoche 41 **********
loss identity:  0.03121667169034481
loss forward:  1.2264845371246338
loss sum:  1.2577011585235596
[0.99973506 0.99973506 0.5042528  0.2536619  0.2536619  0.01592734]
********** Epoche 61 **********
loss identity:  0.032935380935668945
loss forward:  1.3283480405807495
loss sum:  1.3612834215164185
[0.99220026 0.99220026 0.3642573  0.23483764 0.03640407 0.09255002]
********** Epoche 81 **********
loss identity:  0.023897280916571617
loss forward:  1.3087165355682373
loss sum:  1.3326138257980347
[1.0005447  1.0005447  0.31555015 0.31555015 0.132

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▄▄▂▃▅▃▂▃▁▄▄▂▃▄▃▄▃▃▂▂▄▃▂▃▂▃▄▁▃▂▂▃▄▄▁▄▃▂
loss_forward,█▃▄▄▂▄▅▃▂▃▁▄▄▂▃▄▃▄▃▃▃▂▄▃▂▃▂▃▄▁▃▂▂▃▄▄▁▄▃▂
loss_identity,▅▄▃▃▂▃▃▃▅▄▃▄▄▄█▃▅▃▄▃▂▃▂▃▄▄▂▂▆▄▃▇▁▂▃▄▂▂▃▂
epoch,599
loss,1.25465
loss_forward,1.23791
loss_identity,0.01674


Connected to a GPU
********** Epoche 1 **********
loss identity:  0.0597509890794754
loss forward:  1.779867172241211
loss sum:  1.83961820602417
[0.9957807  0.9957807  0.10460124 0.05039342 0.05039342 0.03789065]
********** Epoche 21 **********
loss identity:  0.0616505965590477
loss forward:  1.753999948501587
loss sum:  1.8156505823135376
[1.004217   1.004217   0.01733005 0.01733005 0.07294662 0.07294662]
********** Epoche 41 **********
loss identity:  0.025854449719190598
loss forward:  1.7250511646270752
loss sum:  1.7509056329727173
[1.000318   1.000318   0.08865938 0.08865938 0.01816951 0.04401044]
********** Epoche 61 **********
loss identity:  0.06471572816371918
loss forward:  1.7382997274398804
loss sum:  1.8030154705047607
[1.002128   1.002128   0.02414503 0.12722726 0.12722726 0.11845732]
********** Epoche 81 **********
loss identity:  0.05041515827178955
loss forward:  1.8288872241973877
loss sum:  1.8793023824691772
[0.99945444 0.99945444 0.1093993  0.02409879 0.04046874

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▅▇▇▃▅▅▄▄▁▆▅▅▇▄▆▅▃▅█▄▄▅▅▄▆▂▇▃▄▄▄▇▁▆▆▄▆█▅▆
loss_forward,▄▇▆▃▅▅▄▄▁▆▅▅▇▄▅▆▃▅▇▄▄▄▄▄▅▂▆▃▄▄▃▇▁▆▆▃▆█▅▆
loss_identity,▄▃▅▄▃▅▅▃▃▂▃▄▆▄█▂▂▂▄▃▄▃▄▄▅▂▄▁▁▂▄▃▂▄▄▄▁▃▃▃
epoch,599
loss,1.83614
loss_forward,1.80501
loss_identity,0.03114


Connected to a GPU
********** Epoche 1 **********
loss identity:  0.03199414163827896
loss forward:  1.64121413230896
loss sum:  1.673208236694336
[1.0001674  1.0001674  0.07416733 0.03040887 0.03040887 0.04095354]
********** Epoche 21 **********
loss identity:  0.02037563920021057
loss forward:  1.7225559949874878
loss sum:  1.742931604385376
[1.0008655  1.0008655  0.04386467 0.08282441 0.08282441 0.08438905]
********** Epoche 41 **********
loss identity:  0.023433193564414978
loss forward:  1.7425323724746704
loss sum:  1.7659655809402466
[0.997307   0.997307   0.20895213 0.20895213 0.0638601  0.12756237]
********** Epoche 61 **********
loss identity:  0.020021842792630196
loss forward:  1.738541841506958
loss sum:  1.7585636377334595
[0.9978714  0.9978714  0.02209388 0.05131868 0.03805735 0.03805735]
********** Epoche 81 **********
loss identity:  0.02216717042028904
loss forward:  1.626802921295166
loss sum:  1.648970127105713
[0.99996495 0.99996495 0.10656064 0.02459806 0.06155613